# 01 - Baseline: Clasificación de Spam con scikit-learn

**Objetivo:** construir un primer modelo que clasifique mensajes como *spam* o *ham* (no spam)
usando TF-IDF + Logistic Regression.

## Conceptos clave

- **Dataset:** colección de ejemplos (aquí: mensajes SMS).
- **Feature (X):** lo que el modelo usa como entrada (aquí: el texto del mensaje).
- **Label (y):** la respuesta correcta (aquí: 0=ham, 1=spam).
- **Entrenar:** ajustar el modelo con datos conocidos (train).
- **Evaluar:** medir qué tan bien generaliza con datos nuevos (test).

In [ ]:
import pandas as pd

## 1) Cargar dataset

El archivo `SMSSpamCollection` tiene dos columnas:
- `label`: "ham" o "spam"
- `text`: el mensaje

In [ ]:
df = pd.read_csv(
    "../data/SMSSpamCollection",   # ../ porque estamos en notebooks/
    sep="\t",                      # las columnas están separadas por TAB
    header=None,                   # el archivo no tiene encabezados
    names=["label", "text"]        # nombres de columnas que le asignamos
)

df.head()

## 2) Convertir labels a números

Los modelos necesitan números:
- ham → 0  
- spam → 1

In [ ]:
df["label"] = df["label"].map({"ham": 0, "spam": 1})
df["label"].value_counts()

## 3) Separar Features (X) y Labels (y)

- **X**: textos
- **y**: etiqueta (0 o 1)

In [ ]:
X = df["text"]
y = df["label"]

X.head(), y.head()

## 4) Separar en Train y Test

- Train: para entrenar
- Test: para evaluar como si fueran datos “nuevos”

Usaremos 80% train y 20% test.
También usamos **stratify** para que el porcentaje de spam/ham se mantenga similar en ambos.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

len(X_train), len(X_test)

## 5) Convertir texto a números (TF-IDF) + Modelo

Los modelos no entienden texto directo, necesitan números.

**TF-IDF** convierte el texto en un vector:
- palabras comunes pesan menos
- palabras “importantes” pesan más

Luego entrenamos un modelo de **Logistic Regression** (muy buen baseline para texto).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(lowercase=True)),
    ("clf", LogisticRegression(max_iter=200))
])

model

## 6) Entrenar el modelo

`.fit(X_train, y_train)` significa:
- aprende a relacionar textos (X) con etiquetas (y)

In [ ]:
model.fit(X_train, y_train)

## 7) Predecir con datos de prueba

`.predict(X_test)` genera predicciones (0 o 1) para mensajes que el modelo no vio en entrenamiento.


In [ ]:
y_pred = model.predict(X_test)
y_pred[:10]

## 8) Evaluación del modelo

Usaremos:
- **Precision:** de lo que predije como spam, ¿cuánto era spam real?
- **Recall:** de todo el spam real, ¿cuánto detecté?
- **F1:** balance entre precision y recall

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=["ham", "spam"]))

## 9) Matriz de confusión

Nos muestra:
- Aciertos
- Errores (falsos positivos y falsos negativos)

Es clave porque en spam:
- falso positivo = bloquear un mensaje bueno
- falso negativo = dejar pasar spam

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["ham", "spam"])
disp.plot()
plt.show()

## Ejercicio

1. Escribe 5 mensajes que tú creas que son **spam** y 5 que son **ham**.
2. Usa `model.predict()` para clasificarlos.
3. ¿En cuáles se equivoca el modelo? ¿Qué tipo de error fue (FP o FN)?

## ----------------------------------------------------------------------
## ----------------------------------------------------------------------
## ----------------------------------------------------------------------
## ----------------------------------------------------------------------
## ----------------------------------------------------------------------
## ----------------------------------------------------------------------
## ----------------------------------------------------------------------
## ----------------------------------------------------------------------

In [ ]:
exercise_texts = [
    # 5 HAM (0)
    "Hola, ¿a qué hora nos vemos para la tarea?",
    "Ya llegué a la casa, avísame cuando puedas",
    "¿Me puedes mandar el archivo por favor?",
    "Mañana tengo junta a las 10, ¿vas a poder?",
    "Gracias por tu ayuda, quedamos pendientes",

    # 5 SPAM (1)
    "WIN a FREE iPhone now!!! Click here to claim your prize",
    "Congratulations! You have been selected for a CASH reward. Reply YES",
    "URGENT! Your account has been compromised. Verify here immediately",
    "Get cheap loans approved in minutes! Apply now!",
    "FREE entry to win tickets, text WIN to 12345 now!"
]
true_labels = [0, 0, 0, 0, 0,  1, 1, 1, 1, 1]

pred_labels = model.predict(exercise_texts)
pred_labels

import pandas as pd

results = pd.DataFrame({
    "text": exercise_texts,
    "true_label": true_labels,
    "pred_label": pred_labels
})

# Para leer más fácil: mapear 0/1 a ham/spam
results["true_name"] = results["true_label"].map({0: "ham", 1: "spam"})
results["pred_name"] = results["pred_label"].map({0: "ham", 1: "spam"})

results
